## Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.feature_extraction.text import _VectorizerMixin
from sklearn.feature_selection._base import SelectorMixin
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

### Data Files:
<ul>
    <li>calls.csv: a timeline of outgoing sales calls and the disposition of those calls</li>
    <li>events.csv: any activities that we have on record taking place before the phone calls were made</li>
    <li>companies.csv: the industry and employee count of the companies</li>
    <li>people.csv: the people who were called, along with their job level and function and the ID of the company they work for</li>
    <li>opportunities.csv: the date an opportunity was generated for a contact</li>
</ul>

In [2]:
#import data
events = pd.read_csv("../data/Customer1/events.csv") #contact_id not unique
calls = pd.read_csv("../data/Customer1/calls.csv") #calls not unique
companies = pd.read_csv("../data/Customer1/companies.csv") #company id unique
people = pd.read_csv("../data/Customer1/people.csv")# contact id unique
opportunities = pd.read_csv("../data/Customer1/opportunities.csv")# company id not unique
events = events.rename(columns={"date": "event_date"})
calls = calls.rename(columns={"timestamp": "call_time"})
calls['date'] = pd.to_datetime(calls['date'], format='%Y-%m-%d')

## Data Cleaning
### 1. Calls table
#### Create 'new_calls' table with the following columns: 
<ul>
    <li>Add weekday columns.</li>
    <li>Map positive response of phone calls and calculate the sum.</li>
    <li>Find out max and min duration of each call date. </li>
</ul>

- Unique contact id
- Total Number of calls for each contact_id
- Number of successful calls and non successful calls for each contact
- Most recent call

#### Notes:  

- Successful call = ['MC - Meaningful Conversation', 'MC - Meeting Confirmation', 'MC - MQL/QMA', 'MC - No Need', 'MC - No Authority', 'MC - No Budget', 'MC - No Timeline','MC - No Technical Fit', 'MC - Call Back']
- Unsuccessful call = ['Left Voicemail', 'No Answer', 'Bad Number','Call Back Later', 'Left Live Message','Wrong Point of Contact', 'No Longer w/ Company', 'Discovery Call Scheduled', 'Voicemail- Left Voicemail Drop', 'Call - No Answer']

In [3]:
#sum up the positive responses to call_disposition
# this will take about a minute to run 
new_calls = calls.copy()
del new_calls['date']
new_calls['call_time'] =  pd.to_datetime(new_calls['call_time'])
successful = ['MC - Meaningful Conversation', 'MC - Meeting Confirmation', 'MC - MQL/QMA', 
                     'MC - No Need', 'MC - No Authority', 'MC - No Budget', 'MC - No Timeline', 
                     'MC - No Technical Fit', 'MC - Call Back']
new_calls["call_disposition"] = ["total_success" if x in successful else "total_no_success" for x in new_calls['call_disposition']]

new_calls = new_calls.groupby(["contact_id", "call_disposition"]).count().rename(columns={"call_time":"call_count"}).unstack(level=-1).fillna(0).reset_index()
new_calls.columns = new_calls.columns.droplevel(0)
new_calls = new_calls.rename(columns={"":"contact_id"})
new_calls.columns.name = None
new_calls["total_calls_count"] = new_calls.sum(axis=1)
most_recent = calls[["contact_id", 'call_time']].groupby(['contact_id']).agg(lambda x: max(x)).reset_index()["call_time"]
least_recent = calls[["contact_id", 'call_time']].groupby(['contact_id']).agg(lambda x: min(x)).reset_index()["call_time"]
average_time_between_calls = calls[["contact_id", 'call_time']].groupby(['contact_id']).agg(lambda x: np.mean(pd.Series(sorted(list(pd.to_datetime(x)))).diff().fillna(pd.Timedelta(seconds=0)))).reset_index()["call_time"]
new_calls["most_recent_call"] = pd.to_datetime(most_recent)
new_calls["least_recent_call"] = pd.to_datetime(least_recent)
new_calls["diff_most_least_recent_call"] = new_calls["most_recent_call"] - new_calls["least_recent_call"]
new_calls["average_time_between_calls"] = average_time_between_calls

#### Add binary output 'successful_call' 
Successful call = 1      

Unsuccessful call = 0

In [4]:
new_calls["successful_call"] = [1 if x>=1 else 0 for x in new_calls['total_success']]
new_calls.head()

,contact_id,total_no_success,total_success,total_calls_count,most_recent_call,least_recent_call,diff_most_least_recent_call,average_time_between_calls,successful_call
0,4000047a5b09764c,2.0,0.0,2.0,2019-09-19 21:43:06+00:00,2019-09-19 16:31:08+00:00,0 days 05:11:58,0 days 02:35:59,0
1,400008b800968b87b,5.0,0.0,5.0,2019-10-11 18:43:35+00:00,2019-09-16 18:20:42+00:00,25 days 00:22:53,5 days 00:04:34.600000,0
2,400056858c8a8c7c,3.0,0.0,3.0,2019-10-29 16:12:57+00:00,2019-10-21 22:09:37+00:00,7 days 18:03:20,2 days 14:01:06.666666,0
3,40005c676085ab040,0.0,1.0,1.0,2019-10-28 08:31:29+00:00,2019-10-28 08:31:29+00:00,0 days 00:00:00,0 days 00:00:00,1
4,40006690599488600,1.0,0.0,1.0,2019-12-05 14:16:27+00:00,2019-12-05 14:16:27+00:00,0 days 00:00:00,0 days 00:00:00,0


### 2. People table & Companies table
<ul>
    <li>Merge two table based on contact_id.</li>
    <li>Apply numerical features to job_level (numerical 0 - Other, 1 - Staff ...)</li>
    <li>One-hot encoding based on industry.</li>
</ul>

In [5]:
merged_data = new_calls.copy()
merged_data = merged_data.merge(people, on='contact_id', how='inner')
merged_data = merged_data.drop_duplicates(subset=['contact_id'], keep='first')
merged_data = merged_data.merge(companies, on='company_id', how='inner')
merged_data

,contact_id,total_no_success,total_success,total_calls_count,most_recent_call,least_recent_call,diff_most_least_recent_call,average_time_between_calls,successful_call,job_level,job_function,company_id,industry,employee_range
0,4000047a5b09764c,2.0,0.0,2.0,2019-09-19 21:43:06+00:00,2019-09-19 16:31:08+00:00,0 days 05:11:58,0 days 02:35:59,0,Senior,Information Technology,e8d8ea5956700cd,Retail and Distribution,"1,000 - 4,999"
1,74c45479476ba0b08,1.0,0.0,1.0,2019-12-04 17:47:05+00:00,2019-12-04 17:47:05+00:00,0 days 00:00:00,0 days 00:00:00,0,Manager,Information Technology,e8d8ea5956700cd,Retail and Distribution,"1,000 - 4,999"
2,8bb68486a06a68544,1.0,0.0,1.0,2019-09-19 16:45:02+00:00,2019-09-19 16:45:02+00:00,0 days 00:00:00,0 days 00:00:00,0,Manager,Information Technology,e8d8ea5956700cd,Retail and Distribution,"1,000 - 4,999"
3,400008b800968b87b,5.0,0.0,5.0,2019-10-11 18:43:35+00:00,2019-09-16 18:20:42+00:00,25 days 00:22:53,5 days 00:04:34.600000,0,Manager,Information Technology,398f0747c32be25,Financial Services,"1,000 - 4,999"
4,4494b965497b90960,5.0,0.0,5.0,2019-10-11 18:50:20+00:00,2019-09-16 18:23:23+00:00,25 days 00:26:57,5 days 00:05:23.400000,0,Vice President,Engineering,398f0747c32be25,Financial Services,"1,000 - 4,999"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64379,cccb9ba4766c9a096,10.0,0.0,10.0,2019-07-24 18:29:32+00:00,2019-07-01 22:43:34+00:00,22 days 19:45:58,2 days 06:46:35.800000,0,Other,Other,537215a13e351e3,Business Services,"1,000 - 4,999"
64380,cccba4705550a8bcb,2.0,0.0,2.0,2019-08-28 14:59:20+00:00,2019-08-28 14:57:14+00:00,0 days 00:02:06,0 days 00:01:03,0,C-Level,Engineering,2db9294cb1ede62,Software and Technology,20 - 49
64381,cccc64a9a7ab00a9b,4.0,0.0,4.0,2019-09-15 13:33:31+00:00,2019-08-06 14:38:17+00:00,39 days 22:55:14,9 days 23:43:48.500000,0,C-Level,Management,1e1da1a86a15b87,Recreation,20 - 49
64382,cccc6577647596b96,16.0,0.0,16.0,2019-09-16 15:39:31+00:00,2019-08-09 14:38:04+00:00,38 days 01:01:27,2 days 09:03:50.437500,0,Senior,Business Development,7f50838d4b77767,Software and Technology,"10,000+"


### 3. Events Table
#### Create 'new_events' table with the following columns: 
<ul>
    <li>Find out max and min duration of each event date.</li>
    <li>Calculate the duration of events (last day events - first day events)</li>
</ul>

- Unique contact id
- Total Number of events for each contact_id
- column for each activity_type and the count of that activity
- Date of most recent activity
- Column for month, day of week, week in year, day in month, hour in day, minute

#### Notes:  

- dealing with multiple activities for a single contact_id: keep most recent date event was accessed and the number of times each activity_type was accessed 

In [6]:
events['event_date'] = pd.to_datetime(events['event_date'])
events_new = events[["contact_id", "activity_type", 'event_date']].groupby(["contact_id", "activity_type"]).count().rename(columns={"event_date":"event_count"}).unstack(level=-1).fillna(0).reset_index()
events_new.columns = events_new.columns.droplevel(0)
events_new = events_new.rename(columns={"":"contact_id"})
events_new.columns.name = None
events_new["total_activity_count"] = events_new.sum(axis=1)
most_recent = events[["contact_id", 'event_date']].groupby(['contact_id']).agg(lambda x: max(x)).reset_index()["event_date"]
least_recent = events[["contact_id", 'event_date']].groupby(['contact_id']).agg(lambda x: min(x)).reset_index()["event_date"]
average_time_between_calls = events[["contact_id", 'event_date']].groupby(['contact_id']).agg(lambda x: np.mean(pd.Series(sorted(list(pd.to_datetime(x)))).diff().fillna(pd.Timedelta(seconds=0)))).reset_index()["event_date"]
events_new["most_recent_event"] = pd.to_datetime(most_recent)
events_new["least_recent_event"] = pd.to_datetime(least_recent)
events_new["diff_most_least_recent_event"] = events_new["most_recent_event"] - events_new["least_recent_event"]
events_new["average_time_between_event"] = average_time_between_calls
events_new

,contact_id,attended_event,email_click,email_open,form_fill,known_web_visit,total_activity_count,most_recent_event,least_recent_event,diff_most_least_recent_event,average_time_between_event
0,40008cab9a95789c6,0.0,0.0,0.0,1.0,1.0,2.0,2019-08-20,2019-08-20,0 days,0 days 00:00:00
1,400097495c858cc5b,0.0,0.0,0.0,2.0,0.0,2.0,2019-10-10,2019-10-10,0 days,0 days 00:00:00
2,4000bb6585ca64a04,0.0,0.0,0.0,1.0,3.0,4.0,2019-08-05,2019-08-01,4 days,1 days 00:00:00
3,4004009b076c9a5b0,0.0,0.0,0.0,1.0,1.0,2.0,2019-11-23,2019-11-23,0 days,0 days 00:00:00
4,40044555507458aab,0.0,1.0,0.0,2.0,1.0,4.0,2019-11-06,2019-11-06,0 days,0 days 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...
22246,cccb9ba4766c9a096,0.0,0.0,0.0,1.0,1.0,2.0,2019-06-20,2019-06-20,0 days,0 days 00:00:00
22247,cccc6577647596b96,0.0,0.0,0.0,2.0,6.0,8.0,2019-08-09,2019-07-31,9 days,1 days 03:00:00
22248,cccc6874069b05c88,0.0,0.0,2.0,0.0,0.0,2.0,2019-06-18,2019-03-29,81 days,40 days 12:00:00
22249,cccc6c645ba555604,0.0,0.0,0.0,1.0,1.0,2.0,2019-08-27,2019-08-27,0 days,0 days 00:00:00


In [7]:
merged_data = merged_data.merge(events_new, on='contact_id', how='inner')
merged_data

,contact_id,total_no_success,total_success,total_calls_count,most_recent_call,least_recent_call,diff_most_least_recent_call,average_time_between_calls,successful_call,job_level,...,attended_event,email_click,email_open,form_fill,known_web_visit,total_activity_count,most_recent_event,least_recent_event,diff_most_least_recent_event,average_time_between_event
0,40008cab9a95789c6,0.0,1.0,1.0,2019-08-20 12:50:15+00:00,2019-08-20 12:50:15+00:00,0 days 00:00:00,0 days 00:00:00,1,Other,...,0.0,0.0,0.0,1.0,1.0,2.0,2019-08-20,2019-08-20,0 days,0 days 00:00:00
1,400097495c858cc5b,1.0,0.0,1.0,2019-10-10 09:40:19+00:00,2019-10-10 09:40:19+00:00,0 days 00:00:00,0 days 00:00:00,0,C-Level,...,0.0,0.0,0.0,2.0,0.0,2.0,2019-10-10,2019-10-10,0 days,0 days 00:00:00
2,4000bb6585ca64a04,8.0,0.0,8.0,2019-08-19 15:43:45+00:00,2019-08-01 23:46:21+00:00,17 days 15:57:24,2 days 04:59:40.500000,0,Manager,...,0.0,0.0,0.0,1.0,3.0,4.0,2019-08-05,2019-08-01,4 days,1 days 00:00:00
3,4004009b076c9a5b0,5.0,1.0,6.0,2019-12-03 18:24:16+00:00,2019-11-25 14:42:48+00:00,8 days 03:41:28,1 days 08:36:54.666666,1,Vice President,...,0.0,0.0,0.0,1.0,1.0,2.0,2019-11-23,2019-11-23,0 days,0 days 00:00:00
4,40044555507458aab,1.0,0.0,1.0,2019-11-06 15:22:53+00:00,2019-11-06 15:22:53+00:00,0 days 00:00:00,0 days 00:00:00,0,Staff,...,0.0,1.0,0.0,2.0,1.0,4.0,2019-11-06,2019-11-06,0 days,0 days 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22063,ccc9b60bbcac05a84,2.0,0.0,2.0,2019-12-19 10:02:26+00:00,2019-12-19 09:40:26+00:00,0 days 00:22:00,0 days 00:11:00,0,Manager,...,0.0,0.0,0.0,1.0,1.0,2.0,2019-12-13,2019-12-13,0 days,0 days 00:00:00
22064,ccca446c6979060ab,6.0,1.0,7.0,2019-11-11 20:43:07+00:00,2019-08-08 23:16:17+00:00,94 days 21:26:50,13 days 13:20:58.571428,1,Director,...,0.0,5.0,0.0,0.0,0.0,5.0,2019-10-31,2019-03-27,218 days,43 days 14:24:00
22065,cccb9ba4766c9a096,10.0,0.0,10.0,2019-07-24 18:29:32+00:00,2019-07-01 22:43:34+00:00,22 days 19:45:58,2 days 06:46:35.800000,0,Other,...,0.0,0.0,0.0,1.0,1.0,2.0,2019-06-20,2019-06-20,0 days,0 days 00:00:00
22066,cccc6577647596b96,16.0,0.0,16.0,2019-09-16 15:39:31+00:00,2019-08-09 14:38:04+00:00,38 days 01:01:27,2 days 09:03:50.437500,0,Senior,...,0.0,0.0,0.0,2.0,6.0,8.0,2019-08-09,2019-07-31,9 days,1 days 03:00:00


### Feature Engineering: Number of Days between Event Date and Calls Date

In [8]:
merged_data["most_recent_call_event_diff"] = (merged_data["most_recent_call"].dt.tz_localize(None)-merged_data["most_recent_event"].dt.tz_localize(None))


### Split dates into month, week, dayofweek, day, hour, minute levels

In [9]:
merged_data["most_recent_event_month"] = merged_data["most_recent_event"].dt.month_name()
merged_data["most_recent_event_dayofweek"] = merged_data["most_recent_event"].dt.day_name()
merged_data["most_recent_event_week"] = merged_data["most_recent_event"].dt.week
merged_data["most_recent_event_day"] = merged_data["most_recent_event"].dt.day

merged_data["least_recent_event_month"] = merged_data["least_recent_event"].dt.month_name()
merged_data["least_recent_event_dayofweek"] = merged_data["least_recent_event"].dt.day_name()
merged_data["least_recent_event_week"] = merged_data["least_recent_event"].dt.week
merged_data["least_recent_event_day"] = merged_data["least_recent_event"].dt.day

merged_data["most_recent_call_month"] = merged_data["most_recent_call"].dt.month_name()
merged_data["most_recent_call_dayofweek"] = merged_data["most_recent_call"].dt.day_name()
merged_data["most_recent_call_week"] = merged_data["most_recent_call"].dt.week
merged_data["most_recent_call_day"] = merged_data["most_recent_call"].dt.day
merged_data["most_recent_call_hour"] = merged_data["most_recent_call"].dt.hour

merged_data["least_recent_call_month"] = merged_data["least_recent_call"].dt.month_name()
merged_data["least_recent_call_dayofweek"] = merged_data["least_recent_call"].dt.day_name()
merged_data["least_recent_call_week"] = merged_data["least_recent_call"].dt.week
merged_data["least_recent_call_day"] = merged_data["least_recent_call"].dt.day
merged_data["least_recent_call_hour"] = merged_data["least_recent_call"].dt.hour
merged_data

,contact_id,total_no_success,total_success,total_calls_count,most_recent_call,least_recent_call,diff_most_least_recent_call,average_time_between_calls,successful_call,job_level,...,most_recent_call_month,most_recent_call_dayofweek,most_recent_call_week,most_recent_call_day,most_recent_call_hour,least_recent_call_month,least_recent_call_dayofweek,least_recent_call_week,least_recent_call_day,least_recent_call_hour
0,40008cab9a95789c6,0.0,1.0,1.0,2019-08-20 12:50:15+00:00,2019-08-20 12:50:15+00:00,0 days 00:00:00,0 days 00:00:00,1,Other,...,August,Tuesday,34,20,12,August,Tuesday,34,20,12
1,400097495c858cc5b,1.0,0.0,1.0,2019-10-10 09:40:19+00:00,2019-10-10 09:40:19+00:00,0 days 00:00:00,0 days 00:00:00,0,C-Level,...,October,Thursday,41,10,9,October,Thursday,41,10,9
2,4000bb6585ca64a04,8.0,0.0,8.0,2019-08-19 15:43:45+00:00,2019-08-01 23:46:21+00:00,17 days 15:57:24,2 days 04:59:40.500000,0,Manager,...,August,Monday,34,19,15,August,Thursday,31,1,23
3,4004009b076c9a5b0,5.0,1.0,6.0,2019-12-03 18:24:16+00:00,2019-11-25 14:42:48+00:00,8 days 03:41:28,1 days 08:36:54.666666,1,Vice President,...,December,Tuesday,49,3,18,November,Monday,48,25,14
4,40044555507458aab,1.0,0.0,1.0,2019-11-06 15:22:53+00:00,2019-11-06 15:22:53+00:00,0 days 00:00:00,0 days 00:00:00,0,Staff,...,November,Wednesday,45,6,15,November,Wednesday,45,6,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22063,ccc9b60bbcac05a84,2.0,0.0,2.0,2019-12-19 10:02:26+00:00,2019-12-19 09:40:26+00:00,0 days 00:22:00,0 days 00:11:00,0,Manager,...,December,Thursday,51,19,10,December,Thursday,51,19,9
22064,ccca446c6979060ab,6.0,1.0,7.0,2019-11-11 20:43:07+00:00,2019-08-08 23:16:17+00:00,94 days 21:26:50,13 days 13:20:58.571428,1,Director,...,November,Monday,46,11,20,August,Thursday,32,8,23
22065,cccb9ba4766c9a096,10.0,0.0,10.0,2019-07-24 18:29:32+00:00,2019-07-01 22:43:34+00:00,22 days 19:45:58,2 days 06:46:35.800000,0,Other,...,July,Wednesday,30,24,18,July,Monday,27,1,22
22066,cccc6577647596b96,16.0,0.0,16.0,2019-09-16 15:39:31+00:00,2019-08-09 14:38:04+00:00,38 days 01:01:27,2 days 09:03:50.437500,0,Senior,...,September,Monday,38,16,15,August,Friday,32,9,14


### Convert calculated differences into decimals

In [10]:
merged_data['diff_most_least_recent_call'] = merged_data['diff_most_least_recent_call'].dt.total_seconds()/(60*60*24)
merged_data['average_time_between_calls'] = merged_data['average_time_between_calls'].dt.total_seconds()/(60*60*24)
merged_data['most_recent_call_event_diff'] = merged_data['most_recent_call_event_diff'].dt.total_seconds()/(60*60*24)
merged_data['diff_most_least_recent_event'] = merged_data['diff_most_least_recent_event'].dt.total_seconds()/(60*60*24)
merged_data['average_time_between_event'] = merged_data['average_time_between_event'].dt.total_seconds()/(60*60*24)

<a id='merge'></a>
# Feature: Description

<ul>
<li><b>call_disposition:</b>  sum of the positive response</li>
<li><b>total_calls_count:</b>  total number of calls made</li>
<li><b>most_recent_call:</b>  most recent call made</li>
<li><b>average_time_between_calls:</b>  earliest call made</li>
<li><b>successful_call:</b>  1 - positive response, 0 - negative response</li>
<li><b>job_level:</b>  numerical features of job_level (0 - Other, 1 - Staff, 2 - Manager, 3 - Director, 4 - Senior, 5 - Vice President, 6 - C-Level)</li>
<li><b>job_function:</b> </li>
<li><b>industry:</b> 
       'Aerospace and Defense', 'Agriculture', 'Apparel', 'Associations',
       'Automotive', 'Biotech', 'Business Services', 'Construction',
       'Consumer Goods and Services', 'Education', 'Energy and Utilities',
       'Financial Services', 'Food and Beverage', 'Furniture', 'Government',
       'Hardware', 'Healthcare and Medical', 'Home and Garden',
       'Hospitality and Travel', 'Manufacturing', 'Media and Entertainment',
       'Mining', 'Pharmaceuticals', 'Printing and Publishing', 'Real Estate',
       'Recreation', 'Retail and Distribution', 'Software and Technology',
       'Telecommunications', 'Textiles', 'Transportation and Logistics',
       'success'</li>
<li><b>employee_range:</b> </li>
<li><b>attended_event:</b> number of events attended</li>
<li><b>email_click:</b>  number of emails clicked</li>
<li><b>email_open:</b>  number of emails opened</li>
<li><b>form_fill:</b>  number of forms filled</li>
<li><b>known_web_visit:</b>  number of known website visits</li>  
<li><b>total_activity_count:</b>  total number of activities</li>
<li><b>most_recent_event:</b>  most recent event date</li>
<li><b>least_recent_event:</b>  earliest event date</li>
<li><b>average_time_between_event:</b>  average time between each event</li>
<li><b>most_recent_call_event_diff:</b> time between most recent call and event</li>  
<li><b>most_recent_event_month:</b>  most recent event month</li>
<li><b>most_recent_event_dayofweek:</b>   most recent event day of week </li>
<li><b>most_recent_event_week:</b>   most recent event week (1 to 52)</li>
<li><b>most_recent_event_day:</b>   most recent event day of month</li>
<li><b>least_recent_event_month:</b>  earliest event month</li>
<li><b>most_recent_call_dayofweek:</b>  most recent call day of week </li>
<li><b>most_recent_call_week:</b>  most recent call week (1 to 52)</li>
<li><b>most_recent_call_day:</b>  most recent call day (1 to 52)</li>
<li><b>most_recent_call_hour:</b>  most recent call hour</li>  
<li><b>least_recent_call_month:</b>  most recent call month</li>
<li><b>least_recent_call_week:</b>  most recent call week</li>
<li><b>least_recent_call_day:</b> most recent call day</li>
    
</ul>

In [11]:
merged_data.columns

Index(['contact_id', 'total_no_success', 'total_success', 'total_calls_count',
       'most_recent_call', 'least_recent_call', 'diff_most_least_recent_call',
       'average_time_between_calls', 'successful_call', 'job_level',
       'job_function', 'company_id', 'industry', 'employee_range',
       'attended_event', 'email_click', 'email_open', 'form_fill',
       'known_web_visit', 'total_activity_count', 'most_recent_event',
       'least_recent_event', 'diff_most_least_recent_event',
       'average_time_between_event', 'most_recent_call_event_diff',
       'most_recent_event_month', 'most_recent_event_dayofweek',
       'most_recent_event_week', 'most_recent_event_day',
       'least_recent_event_month', 'least_recent_event_dayofweek',
       'least_recent_event_week', 'least_recent_event_day',
       'most_recent_call_month', 'most_recent_call_dayofweek',
       'most_recent_call_week', 'most_recent_call_day',
       'most_recent_call_hour', 'least_recent_call_month',
       'lea

### Export to CSV

In [12]:
merged_data.to_csv("../data/preprocessed_data.csv")

## Transform Data

In [13]:
numeric_columns = ['total_no_success', 'total_success', 'total_calls_count', 'diff_most_least_recent_call', 
                   'average_time_between_calls', 'attended_event', 'email_click', 
                   'email_open', 'form_fill', 'known_web_visit', 'total_activity_count', 'diff_most_least_recent_event',
                   'average_time_between_event', 'most_recent_call_event_diff']
cat_columns = ['job_level', 'job_function', 'industry', 'employee_range', 'most_recent_call_month', 
                   'least_recent_call_month', 'most_recent_event_month', 'most_recent_event_dayofweek']

numeric_pipeline = make_pipeline(SimpleImputer(strategy='mean'), StandardScaler())
cat_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder())
text_pipeline = make_pipeline(CountVectorizer(), SelectKBest(k=5))

transformers = [
('num', numeric_pipeline, numeric_columns),
('cat', cat_pipeline, cat_columns),
# ('simple_transformer', MinMaxScaler(), ['scalar']),
]

In [14]:
def get_feature_out(estimator, feature_in):
    if hasattr(estimator,'get_feature_names'):
        if isinstance(estimator, _VectorizerMixin):
            # handling all vectorizers
            return [f'vec_{f}' \
                for f in estimator.get_feature_names()]
        else:
            return estimator.get_feature_names(feature_in)
    elif isinstance(estimator, SelectorMixin):
        return np.array(feature_in)[estimator.get_support()]
    else:
        return feature_in


def get_ct_feature_names(ct):
    # handles all estimators, pipelines inside ColumnTransfomer
    # doesn't work when remainder =='passthrough'
    # which requires the input column names.
    output_features = []

    for name, estimator, features in ct.transformers_:
        if name!='remainder':
            if isinstance(estimator, Pipeline):
                current_features = features
                for step in estimator:
                    current_features = get_feature_out(step, current_features)
                features_out = current_features
            else:
                features_out = get_feature_out(estimator, features)
            output_features.extend(features_out)
        elif estimator=='passthrough':
            output_features.extend(ct._feature_names_in[features])
                
    return output_features


In [15]:
all_data = merged_data.copy()
transform_pipe = ColumnTransformer(transformers, remainder='passthrough')
transformed_data = transform_pipe.fit_transform(all_data)
transformed_data = pd.DataFrame(transformed_data, columns=get_ct_feature_names(transform_pipe))

## Export transformed data to csv

In [16]:
transformed_data.to_csv("../data/transformed_data.csv")
transformed_data.head()

,total_no_success,total_success,total_calls_count,diff_most_least_recent_call,average_time_between_calls,attended_event,email_click,email_open,form_fill,known_web_visit,...,least_recent_event_week,least_recent_event_day,most_recent_call_dayofweek,most_recent_call_week,most_recent_call_day,most_recent_call_hour,least_recent_call_dayofweek,least_recent_call_week,least_recent_call_day,least_recent_call_hour
0,-0.563278,0.578702,-0.524504,-0.669884,-0.512907,-0.0963594,-0.270543,-0.457167,-0.410622,-0.151134,...,34,20,Tuesday,34,20,12,Tuesday,34,20,12
1,-0.483506,-0.624841,-0.524504,-0.669884,-0.512907,-0.0963594,-0.270543,-0.457167,0.561967,-0.385351,...,41,10,Thursday,41,10,9,Thursday,41,10,9
2,0.0748994,-0.624841,0.0334576,-0.182789,-0.211168,-0.0963594,-0.270543,-0.457167,-0.410622,0.317298,...,31,1,Monday,34,19,15,Thursday,31,1,23
3,-0.164417,0.578702,-0.12596,-0.445049,-0.327204,-0.0963594,-0.270543,-0.457167,-0.410622,-0.151134,...,47,23,Tuesday,49,3,18,Monday,48,25,14
4,-0.483506,-0.624841,-0.524504,-0.669884,-0.512907,-0.0963594,0.175737,-0.457167,0.561967,-0.151134,...,45,6,Wednesday,45,6,15,Wednesday,45,6,15


In [23]:
list(transformed_data.columns)

['total_no_success',
 'total_success',
 'total_calls_count',
 'diff_most_least_recent_call',
 'average_time_between_calls',
 'attended_event',
 'email_click',
 'email_open',
 'form_fill',
 'known_web_visit',
 'total_activity_count',
 'diff_most_least_recent_event',
 'average_time_between_event',
 'most_recent_call_event_diff',
 'job_level_C-Level',
 'job_level_Director',
 'job_level_Manager',
 'job_level_Other',
 'job_level_Senior',
 'job_level_Staff',
 'job_level_Vice President',
 'job_function_Accounting',
 'job_function_Administrative',
 'job_function_Arts and Design',
 'job_function_Business Development',
 'job_function_Consulting',
 'job_function_Customer Service & Support',
 'job_function_Education',
 'job_function_Engineering',
 'job_function_Finance',
 'job_function_Healthcare Services',
 'job_function_Human Resources',
 'job_function_Information Technology',
 'job_function_Legal',
 'job_function_Management',
 'job_function_Marketing',
 'job_function_Media & Communications',
 '

### Select columns for model

- ['most_recent_event_week', 'most_recent_event_day',
   'least_recent_event_week', 'least_recent_event_day',
   'most_recent_call_week', 'most_recent_call_day',
   'most_recent_call_hour','least_recent_call_week', 
   'least_recent_call_day','least_recent_call_hour'] are categorical variables left as its original value 

In [17]:
model_data = transformed_data.drop(["contact_id", "company_id", "most_recent_event", "least_recent_event", 
                                   "least_recent_event_month", "least_recent_event_dayofweek", 
                                   "most_recent_call_dayofweek", "least_recent_call_dayofweek", 
                                   "most_recent_call", "least_recent_call"], axis=1)

## Export model data to csv

In [18]:
model_data.to_csv("../data/model_data.csv")
model_data.head()

,total_no_success,total_success,total_calls_count,diff_most_least_recent_call,average_time_between_calls,attended_event,email_click,email_open,form_fill,known_web_visit,...,most_recent_event_week,most_recent_event_day,least_recent_event_week,least_recent_event_day,most_recent_call_week,most_recent_call_day,most_recent_call_hour,least_recent_call_week,least_recent_call_day,least_recent_call_hour
0,-0.563278,0.578702,-0.524504,-0.669884,-0.512907,-0.0963594,-0.270543,-0.457167,-0.410622,-0.151134,...,34,20,34,20,34,20,12,34,20,12
1,-0.483506,-0.624841,-0.524504,-0.669884,-0.512907,-0.0963594,-0.270543,-0.457167,0.561967,-0.385351,...,41,10,41,10,41,10,9,41,10,9
2,0.0748994,-0.624841,0.0334576,-0.182789,-0.211168,-0.0963594,-0.270543,-0.457167,-0.410622,0.317298,...,32,5,31,1,34,19,15,31,1,23
3,-0.164417,0.578702,-0.12596,-0.445049,-0.327204,-0.0963594,-0.270543,-0.457167,-0.410622,-0.151134,...,47,23,47,23,49,3,18,48,25,14
4,-0.483506,-0.624841,-0.524504,-0.669884,-0.512907,-0.0963594,0.175737,-0.457167,0.561967,-0.151134,...,45,6,45,6,45,6,15,45,6,15


## Check contact_id column is unique

In [21]:
assert len(transformed_data) == len(transformed_data['contact_id'].unique())